<a href="https://colab.research.google.com/github/chhyyi/aiffel/blob/main/aiffelthon/GK2B_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GK2B L2 outputs crawler
this notebook is created to downloads GK-2B(sattelite) data files from its OPeNDAP database (Maybe it's property of Republic of Korea Government). There is no english description. please see comments in the codeblocks for detail. ___It does not use OPeNDAP request. but python's request module.___


이 노트북은 [nosc.elecocean.co.kr:8080/opendap/GOCI-II/](http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/)에서 GK-2B (천리안 2B) 의  자료를 다운받기 위해 만들었습니다.

## crawler setup: 
시작일, 종료일, 수집된 파일을 저장할 경로

## download jpgs
parameter: 파일명 끝부분(example: 'LA_RI.jpg')  
예시:  
```python
#example : output is directory to save downloaded files.
!mkdir download
crawler=Crawler(start_date='2022/01/01', last_date='2022/01/31', save_dir='download')
crawler.get_files('LA_RI.jpg', 'jpg') 
```

## netcdf (nc file) download

파일명 끝부분
```python
#example
!mkdir ncs
crawler=Crawler(start_date='2022/01/01', last_date='2022/01/31', save_dir='ncs')
crawler.get_files('S007_RI.nc', 'nc')
```     
S007은 슬롯7을 뜻합니다. 참고로 한반도 대부분이 S007에 들어갑니다.
- 슬롯별 위치는 [http://www.khoa.go.kr/nosc/satellite/searchL2.do](http://www.khoa.go.kr/nosc/satellite/searchL2.do) 에서 슬롯 선택을 눌러보세요.


## Description  
### upate log
2022-10-29 jpg downloader, nc file downloader completed

### 접근해야 하는 페이지/파일 구조  
opendap가 directory같은 방식으로 구성돼 있는데, 실제로 우리가 다운받아야 하는 파일은 다음과 같은 주소를 가집니다.   
> http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/01/01/L2/GK2_GC2_L2_20220101_021530/GK2B_GOCI2_L2_20220101_021530_LA_RI.jpg

여기에서, opendap 이후 부분은 파일 시스템의 경로와 같이 생각할수 있고, directory의 내용을 보려면 경로에 있는 'contents.html' 문서를 보면 됩니다. opendap 라이브러리가 있으니 그걸 써도 될텐데 여기선 안씁니다.   
날짜가 정해지면 

>GOCI-II/2022/01/01/L2/  

까지의 경로는 고정돼 있습니다. 여기서 contents.html을 열고 하위 디렉토리를 파악하여

>GOCI-II/2022/01/01/L2/GK2_GC2_L2_20220101_021530/

같은 하위 디렉토리에서 contents.html 을 보면 다운받을 파일 목록이 나타납니다. 이 디렉토리들을 열어 그 안에 있는 이름이 맞는 파일들을 다운받습니다. 사실 nc파일은 html 파일에 들어가서 다운받게 돼 있고, 그 html파일에서 '.html'만 지우면 파일 경로라 그걸로 합니다. 이 주소를 웹브라우저에서 그냥 입력하면 OPeNDAP request를 이용하라고 나옵니다. 그런데 그냥 python 의 request 라이브러리로 해도 다운로드가 되길래 그렇게 합니다. (그러므로 언젠가 막힐 가능성이 있습니다.)

In [9]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import datetime
import urllib.request
import re
import os

import requests
from bs4 import BeautifulSoup

In [110]:
class Crawler():
    def __init__(self, start_date='2021/01/01', last_date='2021/12/31', save_dir='outputs'):
        self.date0=datetime.datetime(*(map(int, start_date.split('/'))))
        self.date1=datetime.datetime(*(map(int, last_date.split('/'))))
        self.dir=save_dir
        self.rootdir='http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/'
    
    def get_parsed_html_soup(self, url):
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.content, 'html.parser')
        return soup

    def sub_path_finder(self, dir, tags, end_with):
        soup=self.get_parsed_html_soup(dir)
        subpaths=soup.select(tags) #same for subdirectory.
        pathlist=[]
        #print('end with:', end_with)
        for line in subpaths:
            #print(line.get('href'), type(line.get('href')), str(end_with in line.get('href')))
            #if type(line.get_)!=str:
            #    pass
            if end_with in line.get('href'):
                pathlist.append(line.get('href'))
        print('pathlist:',pathlist)
        return list(set(pathlist))
    
    def get_files(self, end_with, filetype):
        """get(self, end_with, filetype)
        end_with : 
        filetype : 'jpg' or 'nc'
        """
        date=self.date0
        while date<=self.date1:
            print('\n\nprocessing date {}{:02d}{:02d}'.format(date.year, date.month, date.day))
            #parents of parents directory
            ppdir=self.rootdir+"{:04d}/{:02d}/{:02d}/L2/contents.html".format(date.year, date.month, date.day) 
            sub_dirs=self.sub_path_finder(ppdir, 'tr td a', 'contents.html')
                
            for sub_dir in sub_dirs:
                pdir=re.sub('contents.html$','',ppdir)+sub_dir
                print('searching {} page includes {}'.format(pdir, end_with))

                if filetype=='jpg':
                    filepaths=self.sub_path_finder(pdir, 'tr td a', end_with)
                elif filetype=='nc':
                    filepaths=self.sub_path_finder(pdir, 'td b a', end_with)
                else:
                    print('Crawler.get() got wrong filetype argument')
                    break

                for filename in filepaths:
                    if filetype=='nc':
                        filename=re.sub('.html$', '', filename)

                    file_url=re.sub('contents.html$', '', pdir)+filename
                    save_file_dir=os.path.join(self.dir, filename)
                    print('download file {} as {}'.format(file_url,save_file_dir))
                    urllib.request.urlretrieve(file_url, save_file_dir)
            date+=datetime.timedelta(days=1)
    

In [111]:
#example: download jpgs
crawler=Crawler(start_date='2022/02/04', last_date='2022/02/04', save_dir='download')
crawler.get_files('RI.jpg', 'jpg')



processing date 20220204
pathlist: ['GK2_GC2_L2_20220204_001530/contents.html', 'GK2_GC2_L2_20220204_011530/contents.html', 'GK2_GC2_L2_20220204_021530/contents.html', 'GK2_GC2_L2_20220204_031530/contents.html', 'GK2_GC2_L2_20220204_041530/contents.html', 'GK2_GC2_L2_20220204_051530/contents.html', 'GK2_GC2_L2_20220204_061530/contents.html', 'GK2_GC2_L2_20220204_071530/contents.html', 'GK2_GC2_L2_20220204_081530/contents.html', 'GK2_GC2_L2_20220204_231530/contents.html']
searching http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/02/04/L2/GK2_GC2_L2_20220204_031530/contents.html page includes RI.jpg
pathlist: ['GK2B_GOCI2_L2_20220204_031530_LA_RI.jpg']
download file http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/02/04/L2/GK2_GC2_L2_20220204_031530/GK2B_GOCI2_L2_20220204_031530_LA_RI.jpg as download/GK2B_GOCI2_L2_20220204_031530_LA_RI.jpg
searching http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/02/04/L2/GK2_GC2_L2_20220204_051530/contents.html page includes RI.jpg
pa

In [115]:
#example: download nc files
crawler2=Crawler(start_date='2022/02/04', last_date='2022/02/04', save_dir='ncs')
crawler2.get_files('S007_RI.nc', 'nc')



processing date 20220204
pathlist: ['GK2_GC2_L2_20220204_001530/contents.html', 'GK2_GC2_L2_20220204_011530/contents.html', 'GK2_GC2_L2_20220204_021530/contents.html', 'GK2_GC2_L2_20220204_031530/contents.html', 'GK2_GC2_L2_20220204_041530/contents.html', 'GK2_GC2_L2_20220204_051530/contents.html', 'GK2_GC2_L2_20220204_061530/contents.html', 'GK2_GC2_L2_20220204_071530/contents.html', 'GK2_GC2_L2_20220204_081530/contents.html', 'GK2_GC2_L2_20220204_231530/contents.html']
searching http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/02/04/L2/GK2_GC2_L2_20220204_031530/contents.html page includes S007_RI.nc
pathlist: ['GK2B_GOCI2_L2_20220204_031530_LA_S007_RI.nc.html']
download file http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/02/04/L2/GK2_GC2_L2_20220204_031530/GK2B_GOCI2_L2_20220204_031530_LA_S007_RI.nc as ncs/GK2B_GOCI2_L2_20220204_031530_LA_S007_RI.nc
searching http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/02/04/L2/GK2_GC2_L2_20220204_051530/contents.html page in

In [116]:
!ls ncs -al

total 230916
drwxr-xr-x 3 root root     4096 Oct 29 10:50 .
drwxr-xr-x 1 root root     4096 Oct 29 10:29 ..
-rw-r--r-- 1 root root 16204187 Oct 29 10:31 GK2B_GOCI2_L2_20220203_011530_LA_S007_RI.nc
-rw-r--r-- 1 root root 17728855 Oct 29 10:32 GK2B_GOCI2_L2_20220203_021530_LA_S007_RI.nc
-rw-r--r-- 1 root root 17999040 Oct 29 10:31 GK2B_GOCI2_L2_20220203_031530_LA_S007_RI.nc
-rw-r--r-- 1 root root 17898375 Oct 29 10:32 GK2B_GOCI2_L2_20220203_041530_LA_S007_RI.nc
-rw-r--r-- 1 root root 17187453 Oct 29 10:32 GK2B_GOCI2_L2_20220203_051530_LA_S007_RI.nc
-rw-r--r-- 1 root root 16517823 Oct 29 10:32 GK2B_GOCI2_L2_20220203_061530_LA_S007_RI.nc
-rw-r--r-- 1 root root  9691174 Oct 29 10:32 GK2B_GOCI2_L2_20220203_081630_LA_S007_RI.nc
-rw-r--r-- 1 root root  9856968 Oct 29 10:31 GK2B_GOCI2_L2_20220203_231530_LA_S007_RI.nc
-rw-r--r-- 1 root root 14571912 Oct 29 10:50 GK2B_GOCI2_L2_20220204_011530_LA_S007_RI.nc
-rw-r--r-- 1 root root 14338251 Oct 29 10:50 GK2B_GOCI2_L2_20220204_021530_LA_S007_RI.nc
-r

# supplement
## download web file

In [3]:
import urllib.request

urllib.request.urlretrieve('http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/01/01/L2/GK2_GC2_L2_20220101_011530/GK2B_GOCI2_L2_20220101_011530_LA_RI.nc', 'temp.nc')

('temp.nc', <http.client.HTTPMessage at 0x7fe329451450>)

## how to get next day

In [8]:
#how to get next day - by datetime
date=datetime.datetime(2021,1,1)
date+=datetime.timedelta(days=1)
date

datetime.datetime(2021, 1, 2, 0, 0)

## link extraction
reference: LMS fundamental node, naver news  crawling

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [ ]:
url = 'http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/06/01/L2/GK2_GC2_L2_20220601_001530/contents.html'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.90 Safari/537.36'}
news = requests.get(url, headers=headers)
soup = BeautifulSoup(news.content, 'html.parser')


In [ ]:
jpgs=soup.select("tr td a") #same for subdirectory.
urllist=[]
for line in jpgs:
    urllist.append(line.get('href'))

In [ ]:
for line in urllist:
    if 'RI.jpg' in line:
        print(line)

GK2B_GOCI2_L2_20220601_001530_LA_RI.jpg


In [ ]:
url = 'http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/06/01/L2/GK2_GC2_L2_20220601_001530/contents.html'

In [ ]:
#url : delete "/contents.html"
'/'.join(url.split(sep='/')[:-1])

'http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/06/01/L2/GK2_GC2_L2_20220601_001530'

In [ ]:
ncs=soup.select("td b a")
nclist=[]
for line in ncs:
    nclist.append(line.get('href'))
nclist

In [ ]:
url = 'http://nosc.elecocean.co.kr:8080/opendap/GOCI-II/2022/06/01/L2/'
news = requests.get(url, headers=headers)
soup = BeautifulSoup(news.content, 'html.parser')

In [ ]:
jpgs=soup.select("tr td a") #same for subdirectory.
urllist=[]
for line in jpgs:
    urllist.append(line.get('href'))

In [ ]:
urllist

['GK2_GC2_L2_20220601_001530/contents.html',
 'GK2_GC2_L2_20220601_011530/contents.html',
 'GK2_GC2_L2_20220601_021530/contents.html',
 'GK2_GC2_L2_20220601_031530/contents.html',
 'GK2_GC2_L2_20220601_041530/contents.html',
 'GK2_GC2_L2_20220601_051530/contents.html',
 'GK2_GC2_L2_20220601_061530/contents.html',
 'GK2_GC2_L2_20220601_071530/contents.html',
 'GK2_GC2_L2_20220601_081530/contents.html',
 'GK2_GC2_L2_20220601_231530/contents.html',
 'catalog.xml',
 'http://www.nsf.gov/',
 'http://www.nasa.gov/',
 'http://www.noaa.gov/']

## netCDF4 information

In [117]:
import netCDF4 as nc4
rootgrp = nc4.Dataset('/content/ncs/GK2B_GOCI2_L2_20220204_031530_LA_S007_RI.nc')
print(rootgrp)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6 ACDD-1.3
    acknowledgement: 
    comment: 
    creator_email: kosc@kiost.ac.kr
    creator_name: KOSC
    creator_url: http://kosc.kiost.ac.kr
    earth_radius: 6378137.0
    false_easting: 0.0
    false_northing: 0.0
    geographic_crs_name: WGS 84
    geospatial_lat_max: 39.28563
    geospatial_lat_min: 30.665466
    geospatial_lat_resolution: 250 meters
    geospatial_lat_units: degree_north
    geospatial_lon_max: 130.40538
    geospatial_lon_min: 121.99647
    geospatial_lon_resolution: 250 meters
    geospatial_lon_units: degree_east
    grid_mapping_name: geostationary
    history: input:AC,STATIC__output:RI
    horizontal_datum_name: WGS 1984
    id: GK2B_GOCI2_L2_20220204_031530_LA_S007_RI.nc
    image_center_latitude: 34.785534
    image_center_longitude: 126.338715
    image_lowerright_latitude: 30.66906
    image_lowerright_longitude: 130.15741
    image_upperleft